In [7]:
from psql_helpers import init_db

In [8]:
import asyncio
import uuid
from datetime import datetime, timezone
from psql_models import async_session, User, Meeting, Speaker, DiscussionPoint, UserMeeting, DefaultAccess, AccessLevel

async def create_dummy_data():
    async with async_session() as session:
        # Create users
        users = [
            User(
                email=f"user{i}@example.com",
                username=f"user{i}",
                first_name=f"First{i}",
                last_name=f"Last{i}"
            ) for i in range(1, 4)
        ]
        session.add_all(users)
        await session.flush()

        # Create meetings
        meetings = [
            Meeting(
                transcript="Sample transcript for meeting 1",
            ),
            Meeting(
                transcript="Sample transcript for meeting 2",
            )
        ]
        session.add_all(meetings)
        await session.flush()

        # Create speakers
        speakers = [
            Speaker(name="John Doe"),
            Speaker(name="Jane Smith"),
            Speaker(name="Bob Johnson")
        ]
        session.add_all(speakers)
        await session.flush()

        # Create user meetings (sharing permissions)
        user_meetings = [
            UserMeeting(
                meeting_id=meetings[0].meeting_id,
                user_id=users[0].id,
                access_level=AccessLevel.OWNER.value,
                created_by=users[0].id,
                is_owner=True
            ),
            UserMeeting(
                meeting_id=meetings[0].meeting_id,
                user_id=users[1].id,
                access_level=AccessLevel.TRANSCRIPT.value,
                created_by=users[0].id,
                is_owner=False
            )
        ]
        session.add_all(user_meetings)

        # Create discussion points
        discussion_points = [
            DiscussionPoint(
                summary="Key point 1",
                details="Detailed discussion about point 1",
                referenced_text="Original text reference",
                meeting_id=meetings[0].meeting_id,
                speaker_id=speakers[0].id,
                topic_name="Topic 1",
                topic_type="Decision",
                model="gpt-4"
            ),
            DiscussionPoint(
                summary="Key point 2",
                details="Detailed discussion about point 2",
                referenced_text="Original text reference",
                meeting_id=meetings[0].meeting_id,
                speaker_id=speakers[1].id,
                topic_name="Topic 2",
                topic_type="Action Item",
                model="gpt-4"
            )
        ]
        session.add_all(discussion_points)

        # Create default access permissions
        default_access = [
            DefaultAccess(
                owner_user_id=users[0].id,
                granted_user_id=users[1].id,
                access_level=AccessLevel.TRANSCRIPT.value
            )
        ]
        session.add_all(default_access)

        await session.commit()

In [9]:
#await create_dummy_data()

In [10]:
from psql_helpers import read_table_async

In [11]:
await read_table_async(DiscussionPoint)

,id,summary_index,summary,details,referenced_text,meeting_id,speaker_id,topic_name,topic_type,model
0,1,0,The team successfully resolved a technical iss...,"The specific problem was not detailed, but the...",Ravshan S.K.: Смогли решить проблему.,7343734f-7311-4844-aa5e-86f1f93595c5,1,Problem resolution,solution,MeetingExtraction
1,2,1,Dmitry expressed interest in seeing Ravshan's ...,Dmitry wants to understand how Ravshan approac...,"Dmitry Grankin: Мне интересно посмотреть, как...",7343734f-7311-4844-aa5e-86f1f93595c5,2,Code demonstration,task,MeetingExtraction
2,3,2,The team discussed creating a streaming chat a...,"Ravshan suggested starting with a public API, ...","Dmitry Grankin: Давай, я не знаю, какую-нибуд...",7343734f-7311-4844-aa5e-86f1f93595c5,2,Streaming chat task,task,MeetingExtraction
3,4,3,Ravshan needs to set up access to an API to pr...,Ravshan mentioned the need for an account to a...,"Ravshan S.K.: Да у меня нету айпики, надо там...",7343734f-7311-4844-aa5e-86f1f93595c5,1,API access setup,task,MeetingExtraction
4,5,4,The team plans to integrate real-time weather ...,Dmitry proposed adding real-time weather acces...,Dmitry Grankin: can't provide real-time infor...,7343734f-7311-4844-aa5e-86f1f93595c5,2,Real-time weather access,task,MeetingExtraction
...,...,...,...,...,...,...,...,...,...,...
1124,1124,12,Product Requirement Documents (PRDs) are cruci...,IV stresses that PRDs should be clear and arti...,info vexa: A lot of times the PM job can beco...,b53a31e0-6e27-452e-aa4d-907e67e3d775,56,PRDs,concept,EntityExtraction
1125,1126,14,Anthony Bourdain is referenced by IV regarding...,IV quotes Bourdain to suggest that consuming g...,info vexa: I take inspiration from the camp o...,b53a31e0-6e27-452e-aa4d-907e67e3d775,56,Anthony Bourdain,person,EntityExtraction
1126,1127,15,Vanessa is a friend of IV who helped them deve...,IV mentions Vanessa's advice to read beautiful...,info vexa: I have a friend who's a very good ...,b53a31e0-6e27-452e-aa4d-907e67e3d775,56,Vanessa,person,EntityExtraction
1127,1128,16,Program essays are noted for their succinct an...,IV mentions learning from program essays as a ...,: obviously there's like all the program essa...,b53a31e0-6e27-452e-aa4d-907e67e3d775,56,program essays,topic,EntityExtraction


In [12]:
await read_table_async(UserMeeting)

,id,meeting_id,user_id,access_level,created_at,created_by,is_owner
0,1,7343734f-7311-4844-aa5e-86f1f93595c5,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:22:23.365912+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
1,2,50d5d659-3a57-42f2-8f32-31c7145ebe91,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:22:59.524789+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
2,3,5824bf7d-b528-429f-90fe-7048fa932539,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:24.426869+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
3,4,43fb67c6-9a6b-4fe2-88e1-ced34d0a47e3,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:30.214269+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
4,5,9b4c6346-9510-467e-91f6-3e6fd3bc42c7,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:38.332628+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
...,...,...,...,...,...,...,...
78,79,793fec1a-e8a6-4aff-b185-d5bd1f068463,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,search,2024-10-29 15:59:07.594315+00:00,8b767b85-44fb-4c20-97df-7aa172242273,False
79,80,31f7e0f7-fd0d-4e2c-91fb-e43399af8ce0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,search,2024-10-29 15:59:07.595539+00:00,8b767b85-44fb-4c20-97df-7aa172242273,False
80,81,09c35705-77c8-4f6e-86d3-fa82b9e03a08,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-29 15:59:15.257376+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True
81,82,8d13c8a4-6440-4be0-887b-35708a878b7a,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-29 15:59:18.315069+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True


In [13]:
await read_table_async(Speaker)

,id,name
0,1,RS
1,2,DG
2,3,Ravshan S.K.
3,4,Dmitry Grankin
4,5,Özay Demirezen
5,6,IS
6,7,Ilia Semukhin
7,8,Ilya Semukhin
8,9,Dmitriy Grankin
9,10,nan


In [14]:
await read_table_async(Meeting)

,id,meeting_id,transcript,timestamp
0,1,7343734f-7311-4844-aa5e-86f1f93595c5,"[{'speaker': 'Ravshan S.K.', 'speaker_id': 'TB...",2024-10-25 12:01:02.055999
1,2,50d5d659-3a57-42f2-8f32-31c7145ebe91,"[{'speaker': 'Dmitry Grankin', 'speaker_id': '...",2024-10-24 09:32:32.980000
2,3,5824bf7d-b528-429f-90fe-7048fa932539,"[{'speaker': 'Dmitriy Grankin', 'speaker_id': ...",2024-10-22 17:44:20.910000
3,4,43fb67c6-9a6b-4fe2-88e1-ced34d0a47e3,"[{'speaker': 'TBD', 'speaker_id': 'TBD', 'cont...",2024-10-22 11:22:45.390000
4,5,9b4c6346-9510-467e-91f6-3e6fd3bc42c7,"[{'speaker': 'Dmitriy Grankin', 'speaker_id': ...",2024-10-21 11:48:48.048000
...,...,...,...,...
72,73,793fec1a-e8a6-4aff-b185-d5bd1f068463,"[{'speaker': 'Olga Nemirovskaya', 'speaker_id'...",2024-10-11 14:28:15.060000
73,74,31f7e0f7-fd0d-4e2c-91fb-e43399af8ce0,"[{'speaker': 'Olga Nemirovskaya', 'speaker_id'...",2024-10-10 15:47:46.824000
74,75,09c35705-77c8-4f6e-86d3-fa82b9e03a08,"[{'speaker': 'Olga Nemirovskaya', 'speaker_id'...",2024-10-10 15:32:31.844000
75,76,8d13c8a4-6440-4be0-887b-35708a878b7a,"[{'speaker': 'TBD', 'speaker_id': 'TBD', 'cont...",2024-10-09 13:28:48.700000


In [15]:
await read_table_async(DefaultAccess)

,owner_user_id,granted_user_id,access_level,created_at
0,8b767b85-44fb-4c20-97df-7aa172242273,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,search,2024-10-29 15:59:07.578257+00:00


In [16]:
await read_table_async(User)

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,2280905@gmail.com,Dmitriy Grankin,Dmitriy,Grankin,https://lh3.googleusercontent.com/a/ACg8ocLu4Z...,2024-10-29 15:22:09.756982+00:00,2024-10-29 15:22:09.756986+00:00
1,8b767b85-44fb-4c20-97df-7aa172242273,olganemirovskaya@gmail.com,Olga Nemirovskaya,Olga,Nemirovskaya,https://lh3.googleusercontent.com/a/ACg8ocKR1i...,2024-10-29 15:55:22.471548+00:00,2024-10-29 15:55:22.471552+00:00


In [20]:
df = await read_table_async(UserMeeting)
df['access_level'].unique()

array(['owner', 'search'], dtype=object)

In [ ]:
Speaker